In [1]:
!pip install termcolor
!pip install pandas_ta

In [2]:
from kiteconnect import KiteConnect
import urllib.parse
import tqdm
from requests import auth 
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import os, time
from os.path import expanduser
from selenium.webdriver.chrome.options import Options
import pyotp
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
import pdb

import sys
import hmac, base64, struct, hashlib, time
import requests


zerodha_cred = {
    'User':{
        'user_id' : 'RK2267',
        'password': "Shan@#2021",
        'api_key' : "pv2830q1vbrhu1eu",   
        'common_ans' : "",
        'api_secret' :'gz426yatawim9xruj2p6e51c40k44r7n',  
        'auth_key': 'TOOAIR75STSUWM67LS3NVUX7B2LGKJ7R'
    },

}
my_api_key=zerodha_cred['User']['api_key']

'''def telegram_bot_sendtext(my_message='SYSTEM RUNNING \n ------------------------- \nNo input received from program'):
    bot_token = '2020880061:AAHDR4kpcb4Y7lsbq59uUpgcqq7hiHYK_AE'  # ParkingBot ID
    bot_chatID = '-746818656'  # TelegramGroup ID 746818656

    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + my_message
    response = requests.get(send_text)
    return response.json()

telegram_bot_sendtext("Starting Zerodha Auth Key fetching")'''


def get_hotp_token(secret, intervals_no):
    key = base64.b32decode(secret, True)
    msg = struct.pack(">Q", intervals_no)
    h = hmac.new(key, msg, hashlib.sha1).digest()
    o = h[19] & 15
    h = (struct.unpack(">I", h[o:o+4])[0] & 0x7fffffff) % 1000000
    return h

def get_totp_token(secret):
    totp = str(get_hotp_token(secret, intervals_no=int(time.time())//30))
    if len(totp) < 6:
        totp = '0' + totp
    return totp


def getkite(_client):

    user_id = zerodha_cred[_client]['user_id']
    password = zerodha_cred[_client]['password']
    api_key = zerodha_cred[_client]['api_key']
    kite = KiteConnect(api_key=api_key)
    # common_ans = zerodha_cred[_client]['common_ans']
    auth_key = zerodha_cred[_client]['auth_key']
    url  = 'https://kite.trade/connect/login?v=3&api_key='+api_key 
    # telegram_bot_sendtext("Line 92")

    try:
        options = Options()
        options.add_argument('--no-sandbox')
        options.add_argument('--headless')
        options.add_argument('--disble-dev-shm-usage')
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get(url)
        time.sleep(2)
        input_fields = driver.find_elements_by_tag_name('input')
        input_fields[0].send_keys(user_id)
        input_fields[1].send_keys(password)
        input_fields[1].send_keys(Keys.ENTER)
        time.sleep(1)
        two_fa = driver.find_elements_by_tag_name('input')
        common_ans = get_totp_token(auth_key)
        two_fa[0].send_keys(str(common_ans))
        two_fa[0].send_keys(Keys.ENTER)
        time.sleep(1)
        while("request_token=" not in driver.current_url):
            print (driver.current_url)
            driver.save_screenshot('login.png')
            time.sleep(3)
        redirect_url = driver.current_url
        #print('redirectURL-',redirect_url)
        parsed = urllib.parse.urlparse(redirect_url)
        query_dict = dict(urllib.parse.parse_qsl(parsed.query))
        request_token = query_dict['request_token']
        driver.close
        return request_token
    except Exception as e:
        print('closing')
        driver.close()
def gen_acc_token():
    global acc_key
    users = ['User'] 
    auth_keys={}
    cur_date=datetime.now().strftime("%y_%m_%d")

    for _client in users:
        api_key = zerodha_cred[_client]['api_key']
        api_secret = zerodha_cred[_client]['api_secret']

        req_token=getkite(_client)  
        # req_token = '8DoJtcPaDkOEj0bxCOm4KC7u4Y3QUd8H'
        url  = 'https://kite.trade/connect/login?v=3&api_key='+api_key 
        print ('url-',url)
        # pdb.set_trace()
        # telegram_bot_sendtext("Line 186")
        # pdb.set_trace()
        kite = KiteConnect(api_key=api_key)
        data = kite.generate_session(req_token, api_secret=api_secret)
        access_token=data["access_token"]
        print(data['user_name'],'has successfully signed in.')
        acc_key=access_token
        print('Access_token->',access_token)
        auth_keys[_client] = access_token

        auth_keys['date'] = cur_date
        telegram_bot_sendtext("*Auth keys extracted!*\n\nUser = \t\t" + _client   +  "\nAuth Key = \t" + auth_keys[_client] +  "\nDate = \t" + auth_keys['date'])

if __name__ == '__main__':
    try:
        gen_acc_token()
        acc_key=""
    except Exception as e:
        print('Happy Trading !')
        #print(my_api_key,acc_key)
        #telegram_bot_sendtext(f" Error in Token Generation - {str(e)}")

url- https://kite.trade/connect/login?v=3&api_key=pv2830q1vbrhu1eu
Kakal Krishna Rao has successfully signed in.
Access_token-> 0zbanc5UBbmP5dsgDuTnBPoGCi458BU6
Happy Trading !


In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 55em; }</style>"))

In [4]:
from datetime import *

def history_data_cpr(ticker_val):
    data1=['' for i in range(1)]
    start_date=(datetime.now()-timedelta(days=1029))
    new_date=(datetime.now())
    
    startd=pd.to_datetime(str(start_date)[:10])
    endd=pd.to_datetime(str(new_date))
    #print(startd,endd)
    
    url1="https://api.kite.trade/instruments/historical/"+str(ticker_val)+"/day?from="+str(startd)+"&to="+str(endd)
    #print(url1)
    HEADERS = {"X-Kite-Version": "3", "Authorization":"token pv2830q1vbrhu1eu:"+acc_key}
    res1 = requests.get(url1, headers=HEADERS)
    i=0
    data1[i] = res1.json()
    data1[i] = data1[i]["data"]["candles"]
    
    data1[i]=pd.DataFrame(data1[i])
    
    data1[i] = data1[i].rename(columns={0: 'Time', 1: 'Open', 2:'High', 3:'Low', 4:'Close', 5:'Volume'})
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    #print('History_data {} collected .'.format(ticker_val))
    return data1

In [5]:
from datetime import datetime, timedelta
data={}

def cpr_day(df):
    i=0
    enter=False
    k=1
    m=0
    global data
    data={0:{0:'1',1:'1',2:'1'}}
    x=0
    year=2020
    while(i<len(df)):
        
        if pd.to_datetime(df.Time.iloc[i]).year==year+1:
            #print('NEW-YEAR')
            k=1
            m=0
            year=year+1
            enter=False
           
        
        if pd.to_datetime(df.Time.iloc[i]).month==k and not enter:
            start=pd.to_datetime(df['Time'].iloc[i])
            m=k+1
            enter=True
            low=float('inf')
            high=float('-inf')
            low=min(low,df['Low'].iloc[i])
            high=max(high,df['High'].iloc[i])
            #print('Enter',df.Time.iloc[i],m,k)
            
        if enter and pd.to_datetime(df.Time.iloc[i]).month!=m:
            #print('on',df.Time.iloc[i])
            low=min(low,df['Low'].iloc[i])
            high=max(high,df['High'].iloc[i])
                   
            
        if enter and pd.to_datetime(df.Time.iloc[i]).month==m:
            k+=1
            m+=1
            i-=1
            enter=False
            #print('final',df.Time.iloc[i])
            low=min(low,df['Low'].iloc[i])
            high=max(high,df['High'].iloc[i])
            close=df['Close'].iloc[i]
            pivot=(high+low+close)/3
            bc=(high+low)/2
            tc=(pivot-bc)+pivot
            dt=str(start)
            st=dt[:10]+'T'+'00:00:00+0530'
            km={'Start':st,'End':df['Time'].iloc[i],'Pivot':pivot,'BC':bc,'TC':tc}
            #print(km)
            data[x]=km
            x+=1
            
        if pd.to_datetime(df.Time.iloc[i]).month==k and enter and df.Time.iloc[i]==df.Time.iloc[-1]:
            low=min(low,df['Low'].iloc[i])
            high=max(high,df['High'].iloc[i])
            close=df['Close'].iloc[i]
            pivot=(high+low+close)/3
            bc=(high+low)/2
            tc=(pivot-bc)+pivot
            dt=str(start)
            st=dt[:10]+'T'+'00:00:00+0530'
            km={'Start':st,'End':df['Time'].iloc[i],'Pivot':pivot,'BC':bc,'TC':tc}
            #print('R',km)
            data[x]=km
            x+=1
            
        
        i+=1

In [6]:
#%%capture cap --no-stderr

from termcolor import colored
import pandas_ta as ta
import pandas as pd
from datetime import *

cpr_1day={}
cpr_1day_dict={}

def crossover(dfx,tkname,dx):
   
    dfx['Pivot']=0.0
    dfx['BC']=0.0
    dfx['TC']=0.0
    enter=False
    k=1
    x=""
    for i in range(1,len(dfx)):
        
        if k==len(data):
            break
        x=dx[k]['Start']
        
        if str(dfx['Time'].iloc[i])==x:
            #print('Enter',x)
            enter=True
            dfx['Pivot'].iloc[i]=dx[k-1]['Pivot']
            dfx['BC'].iloc[i]=dx[k-1]['BC']
            dfx['TC'].iloc[i]=dx[k-1]['TC']
        if enter and dfx['Time'].iloc[i]==dx[k]['End']:
            #print('last',dfx['Time'].iloc[i])
            dfx['Pivot'].iloc[i]=dx[k-1]['Pivot']
            dfx['BC'].iloc[i]=dx[k-1]['BC']
            dfx['TC'].iloc[i]=dx[k-1]['TC']
            #print(dfx['Pivot'].iloc[i],dfx['BC'].iloc[i])
            k+=1
            enter=False
        elif enter and dfx['Time'].iloc[i]!=dx[k]['End']:
            x=""
            #print('wk',dfx['Time'].iloc[i])
            dfx['Pivot'].iloc[i]=dx[k-1]['Pivot']
            dfx['BC'].iloc[i]=dx[k-1]['BC']
            dfx['TC'].iloc[i]=dx[k-1]['TC']
            #print(dfx['Pivot'].iloc[i],dfx['BC'].iloc[i])
    
    #cpr_1day[tkname]=df
    k={}
    k['Close']=dfx['Close'].iloc[-1]
    k['Pivot']=dfx['Pivot'].iloc[-1]
    k['BC']=dfx['BC'].iloc[-1]
    k['TC']=dfx['TC'].iloc[-1]
    
    cpr_1day_dict[tkname]=k
            

    
    
       
    #print('------------------------------------------------------------------------------------------------------------')
    
    
cpr_data={}
#dicct={4267265: 'BAJAJ-AUTO'}
dicct={5633: 'ACC', 6401: 'ADANIENT', 3861249: 'ADANIPORTS', 325121: 'AMBUJACEM', 40193: 'APOLLOHOSP', 54273: 'ASHOKLEY', 60417: 'ASIANPAINT', 1510401: 'AXISBANK', 4267265: 'BAJAJ-AUTO', 4268801: 'BAJAJFINSV', 81153: 'BAJFINANCE', 579329: 'BANDHANBNK', 1195009: 'BANKBARODA', 98049: 'BEL', 108033: 'BHARATFORG', 2714625: 'BHARTIARTL', 112129: 'BHEL', 140033: 'BRITANNIA', 2763265: 'CANBK', 177665: 'CIPLA', 5215745: 'COALINDIA', 1215745: 'CONCOR', 486657: 'CUMMINSIND', 5105409: 'DEEPAKNTR', 2800641: 'DIVISLAB', 3771393: 'DLF', 225537: 'DRREDDY', 232961: 'EICHERMOT', 261889: 'FEDERALBNK', 315393: 'GRASIM', 589569: 'HAL', 1850625: 'HCLTECH', 340481: 'HDFC', 341249: 'HDFCBANK', 119553: 'HDFCLIFE', 345089: 'HEROMOTOCO', 348929: 'HINDALCO', 356865: 'HINDUNILVR', 1270529: 'ICICIBANK', 2863105: 'IDFCFIRSTB', 387073: 'INDHOTEL', 1346049: 'INDUSINDBK', 408065: 'INFY', 3484417: 'IRCTC', 424961: 'ITC', 1723649: 'JINDALSTEL', 3001089: 'JSWSTEEL', 4632577: 'JUBLFOOD', 492033: 'KOTAKBANK', 511233: 'LICHSGFIN', 2939649: 'LT', 4561409: 'LTI', 519937: 'M&M', 2815745: 'MARUTI', 582913: 'MRF', 6054401: 'MUTHOOTFIN', 2977281: 'NTPC', 633601: 'ONGC', 2730497: 'PNB', 3834113: 'POWERGRID', 4708097: 'RBLBANK', 738561: 'RELIANCE', 758529: 'SAIL', 4600577: 'SBICARD', 779521: 'SBIN', 794369: 'SHREECEM', 837889: 'SRF', 857857: 'SUNPHARMA', 871681: 'TATACHEM', 884737: 'TATAMOTORS', 877057: 'TATAPOWER', 895745: 'TATASTEEL', 2953217: 'TCS', 3465729: 'TECHM', 897537: 'TITAN', 2170625: 'TVSMOTOR', 2952193: 'ULTRACEMCO', 2889473: 'UPL', 784129: 'VEDL', 951809: 'VOLTAS', 969473: 'WIPRO', 975873: 'ZEEL'}
tokens=list(dicct.keys())
for tk in tokens:
    df=history_data_cpr(tk)[0]
    cpr_day(df)
    cpr_data[tk]=data[list(data.keys())[-2]]
    crossover(df,dicct[tk],data)

In [7]:
from datetime import *
def history_data(ticker_val):
    data1=['' for i in range(1)]
    t2=(datetime.now() - timedelta(days=0))
    t1=(datetime.now() - timedelta(days=1))
    curr_hr=str(t2).split(':')[0].split(' ')[-1]
    curr_min=str(t2).split(':')[1]
    curr_secs=str(t2).split(':')[2]
    final=curr_hr+':'+curr_min+':'+curr_secs

    t1=str(t1)
    t2=str(t2)
    t1=t1.split(" ")
    t3=t1[1].split(':')
    t3[0]='9'
    t3[1]='00'
    t3=t3[0]+':'+t3[1]+':'+t3[2]
    t1=t1[0]+"+"+t3
    t2=t2.split(" ")
    t3=t2[1].split(':')
    t3[0]='9'
    t3[1]='00'
    t3=t3[0]+':'+t3[1]+':'+t3[2]
    t2=t2[0]+"+"+final
    t1,t2

    url1="https://api.kite.trade/instruments/historical/"+str(ticker_val)+"/minute?from="+t1+"&to="+t2
    #print(url1)
    HEADERS = {"X-Kite-Version": "3", "Authorization":"token pv2830q1vbrhu1eu:"+acc_key}
    res1 = requests.get(url1, headers=HEADERS)
    i=0
    data1[i] = res1.json()
    data1[i] = data1[i]["data"]["candles"]
    data1[i]=pd.DataFrame(data1[i])
    data1[i] = data1[i].rename(columns={0: 'Time', 1: 'Open', 2:'High', 3:'Low', 4:'Close', 5:'Volume'})
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    #print('History_data {} collected .'.format(ticker_val))
    return data1

In [9]:
import threading
import logging
from kiteconnect import KiteTicker
from termcolor import colored
import time
import tqdm
import pandas as pd
from datetime import datetime
from datetime import timedelta
from multiprocessing.pool import ThreadPool

rex=set()
mx=set()
st=set()
kws = KiteTicker(my_api_key, acc_key)
#dicct={4267265: 'BAJAJ-AUTO'}
dicct={5633: 'ACC', 6401: 'ADANIENT', 3861249: 'ADANIPORTS', 325121: 'AMBUJACEM', 40193: 'APOLLOHOSP', 54273: 'ASHOKLEY', 60417: 'ASIANPAINT', 1510401: 'AXISBANK', 4267265: 'BAJAJ-AUTO', 4268801: 'BAJAJFINSV', 81153: 'BAJFINANCE', 579329: 'BANDHANBNK', 1195009: 'BANKBARODA', 98049: 'BEL', 108033: 'BHARATFORG', 2714625: 'BHARTIARTL', 112129: 'BHEL', 140033: 'BRITANNIA', 2763265: 'CANBK', 177665: 'CIPLA', 5215745: 'COALINDIA', 1215745: 'CONCOR', 486657: 'CUMMINSIND', 5105409: 'DEEPAKNTR', 2800641: 'DIVISLAB', 3771393: 'DLF', 225537: 'DRREDDY', 232961: 'EICHERMOT', 261889: 'FEDERALBNK', 315393: 'GRASIM', 589569: 'HAL', 1850625: 'HCLTECH', 340481: 'HDFC', 341249: 'HDFCBANK', 119553: 'HDFCLIFE', 345089: 'HEROMOTOCO', 348929: 'HINDALCO', 356865: 'HINDUNILVR', 1270529: 'ICICIBANK', 2863105: 'IDFCFIRSTB', 387073: 'INDHOTEL', 1346049: 'INDUSINDBK', 408065: 'INFY', 3484417: 'IRCTC', 424961: 'ITC', 1723649: 'JINDALSTEL', 3001089: 'JSWSTEEL', 4632577: 'JUBLFOOD', 492033: 'KOTAKBANK', 511233: 'LICHSGFIN', 2939649: 'LT', 4561409: 'LTI', 519937: 'M&M', 2815745: 'MARUTI', 582913: 'MRF', 6054401: 'MUTHOOTFIN', 2977281: 'NTPC', 633601: 'ONGC', 2730497: 'PNB', 3834113: 'POWERGRID', 4708097: 'RBLBANK', 738561: 'RELIANCE', 758529: 'SAIL', 4600577: 'SBICARD', 779521: 'SBIN', 794369: 'SHREECEM', 837889: 'SRF', 857857: 'SUNPHARMA', 871681: 'TATACHEM', 884737: 'TATAMOTORS', 877057: 'TATAPOWER', 895745: 'TATASTEEL', 2953217: 'TCS', 3465729: 'TECHM', 897537: 'TITAN', 2170625: 'TVSMOTOR', 2952193: 'ULTRACEMCO', 2889473: 'UPL', 784129: 'VEDL', 951809: 'VOLTAS', 969473: 'WIPRO', 975873: 'ZEEL'}
tokens=list(dicct.keys())

url='https://api.telegram.org/bot5761331728:AAEeP9GPp69JUYXwtfw5ysP8HwugoHXZdGw/sendMessage?chat_id=-665312475&text='
'''tokens=[60750087,59897095,60128519]
dicct={60750087:'SILVERMIC',59897095:'SILVERM',60128519:'SILVER'}'''
flag=False
x=-1
x2=-1
s=""
count=0

for i in tqdm.tqdm(tokens):
    prev=history_data_cpr(i)[0]
    globals()['prevclose'+'_{}'.format(i)]=prev['Close'].iloc[-2]
    
    
            
def calculate(tkname,s,ltp,ohlc):
    global rex,x,dicct,tokens,mx,flag,st,cpr_1day_dict
    cpx=cpr_1day_dict[dicct[tkname]]
    #print(tx)
    if x^(s.minute)==0:
        if (s.second==59 or s.second==58) and tkname not in st:
            st.add(tkname)
            res={}
            res={'Ticker':dicct[tkname],'Time':s,'Close':ltp,'Pivot':cpx['Pivot'],'prev':globals()['prevclose'+'_{}'.format(tkname)]}
            print(res)
            if globals()['prevclose'+'_{}'.format(tkname)] < max(cpx['TC'],cpx['BC']) and ltp>max(cpx['TC'],cpx['BC']):
                print('B Prev:',globals()['prevclose'+'_{}'.format(tkname)],'max',max(cpx['TC'],cpx['BC']),'Close',ltp)
                requests.get(url+'TEST-> BUY {} at {}'.format(dicct[tkname],ltp),stream=True)

            elif ltp<min(cpx['TC'],cpx['BC']) and globals()['prevclose'+'_{}'.format(tkname)]>max(cpx['TC'],cpx['BC']):
                requests.get(url+'TEST-> SELL {} at {}'.format(dicct[tkname],ltp),stream=True)
                print('S Prev:',globals()['prevclose'+'_{}'.format(tkname)],'max',max(cpx['TC'],cpx['BC']),'Close',ltp)

            else:
                print('N Prev:',globals()['prevclose'+'_{}'.format(tkname)],'max',max(cpx['TC'],cpx['BC']),'Close',ltp)


            print()

    else:
        x=s.minute
        #globals()['prevclose'+'_{}'.format(tkname)]=ltp
        flag=False
        st=set()
        print("-------------------------START OF {} mins-----------------------".format(x))
    
    

            
def on_ticks(ws, ticks):
    logging.debug("Ticks: {}".format(ticks))
    tim=-1
    thread_list=[]
    for turn in range(len(ticks)):
        ltp=ticks[turn]['last_price']
        ohlc=ticks[turn]['ohlc']
        T2=datetime.now()
        hr2=int(T2.hour)
        min2=int(T2.minute)
        sec2=int(T2.second)
        tk=ticks[turn]['instrument_token']
        #T4=str(T2).split(' ')[0]
        TR1=str(T2).split(' ')[1]
        s=pd.to_datetime(TR1)
        try:
            t1=threading.Thread(target=calculate,args=[tk,s,ltp,ohlc])
            #t2=threading.Thread(target=calculate,args=[tk,s,ltp,ohlc])
            t1.start()
            t1.join()
        except:
            print('Error1')
            continue
        
        
def on_connect(ws, response):
    print("Trading starting soon....")
    check_flag=False
    ws.subscribe(tokens)
    ws.set_mode(ws.MODE_FULL, tokens)
    print("Daily Trading Started. Good Luck....")
    requests.get(url+'Daily Trading Started',stream=True)
          
            
def on_close(ws, code, reason):

    # kws.enable_reconnect(reconnect_interval=5, reconnect_tries=50)
    # print("reconnecting...")
    requests.get(url+'Closing Connection',stream=True)
    print("closing connection")
    logging.info("Connection closed: {code} - {reason}".format(code=code, reason=reason))
    # ws.stop()

def on_error(ws, code, reason):
    logging.info("123456Connection error: {code} - {reason}".format(code=code, reason=reason))


def on_reconnect(ws, attempts_count):
    logging.info("123456Reconnecting: {}".format(attempts_count))


def on_noreconnect(ws):
    logging.info("123456Reconnect failed.")


# Assign the callbacks.
kws.on_ticks = on_ticks

kws.on_close = on_close
kws.on_error = on_error
kws.on_connect = on_connect
kws.on_reconnect = on_reconnect
kws.on_noreconnect = on_noreconnect
kws.connect(threaded=True, disable_ssl_verification=False)


100%|███████████████████████████████████████████| 82/82 [00:37<00:00,  2.20it/s]


Trading starting soon....
Daily Trading Started. Good Luck....
-------------------------START OF 59 mins-----------------------
{'Ticker': 'ZEEL', 'Time': Timestamp('2022-12-06 13:59:59.676672'), 'Close': 264.6, 'Pivot': 260.21666666666664, 'prev': 268.2}
N Prev: 268.2 max 262.4083333333333 Close 264.6

{'Ticker': 'BAJAJ-AUTO', 'Time': Timestamp('2022-12-06 13:59:59.677620'), 'Close': 3675.6, 'Pivot': 3718.1166666666663, 'prev': 3659.9}
N Prev: 3659.9 max 3734.408333333333 Close 3675.6

{'Ticker': 'COALINDIA', 'Time': Timestamp('2022-12-06 13:59:59.678023'), 'Close': 231.05, 'Pivot': 239.0, 'prev': 231.25}
N Prev: 231.25 max 244.875 Close 231.05

{'Ticker': 'HDFC', 'Time': Timestamp('2022-12-06 13:59:59.678374'), 'Close': 2668.1, 'Pivot': 2624.8833333333337, 'prev': 2675.8}
N Prev: 2675.8 max 2658.716666666667 Close 2668.1

{'Ticker': 'HDFCBANK', 'Time': Timestamp('2022-12-06 13:59:59.678705'), 'Close': 1608.45, 'Pivot': 1580.0166666666667, 'prev': 1612.95}
N Prev: 1612.95 max 1594.233

{'Ticker': 'AXISBANK', 'Time': Timestamp('2022-12-06 14:00:59.041704'), 'Close': 899.5, 'Pivot': 882.4499999999999, 'prev': 899.2}
N Prev: 899.2 max 891.8499999999999 Close 899.5

{'Ticker': 'HINDUNILVR', 'Time': Timestamp('2022-12-06 14:00:59.047684'), 'Close': 2641.25, 'Pivot': 2603.4833333333336, 'prev': 2619.85}
N Prev: 2619.85 max 2643.916666666667 Close 2641.25

{'Ticker': 'IRCTC', 'Time': Timestamp('2022-12-06 14:00:59.048296'), 'Close': 716.8, 'Pivot': 740.5666666666666, 'prev': 729.1}
N Prev: 729.1 max 742.75 Close 716.8

{'Ticker': 'BHARATFORG', 'Time': Timestamp('2022-12-06 14:00:59.050115'), 'Close': 839.45, 'Pivot': 857.4166666666666, 'prev': 847.45}
N Prev: 847.45 max 858.4833333333332 Close 839.45

{'Ticker': 'HDFCLIFE', 'Time': Timestamp('2022-12-06 14:00:59.053670'), 'Close': 587.6, 'Pivot': 569.5500000000001, 'prev': 588.9}
N Prev: 588.9 max 580.0750000000002 Close 587.6

{'Ticker': 'TATAPOWER', 'Time': Timestamp('2022-12-06 14:00:59.055543'), 'Close': 228.0, 'Pivot':


{'Ticker': 'ASIANPAINT', 'Time': Timestamp('2022-12-06 14:02:59.429307'), 'Close': 3150.4, 'Pivot': 3136.2000000000003, 'prev': 3156.9}
N Prev: 3156.9 max 3155.6750000000006 Close 3150.4

{'Ticker': 'ITC', 'Time': Timestamp('2022-12-06 14:02:59.431518'), 'Close': 337.0, 'Pivot': 345.2833333333333, 'prev': 336.75}
N Prev: 336.75 max 347.92499999999995 Close 337.0

{'Ticker': 'ONGC', 'Time': Timestamp('2022-12-06 14:02:59.432306'), 'Close': 141.7, 'Pivot': 139.26666666666668, 'prev': 143.75}
N Prev: 143.75 max 140.18333333333334 Close 141.7

{'Ticker': 'VEDL', 'Time': Timestamp('2022-12-06 14:02:59.433037'), 'Close': 313.9, 'Pivot': 303.31666666666666, 'prev': 319.25}
N Prev: 319.25 max 303.8333333333333 Close 313.9

{'Ticker': 'WIPRO', 'Time': Timestamp('2022-12-06 14:02:59.433806'), 'Close': 410.8, 'Pivot': 400.2833333333333, 'prev': 412.35}
N Prev: 412.35 max 403.5916666666666 Close 410.8

{'Ticker': 'HEROMOTOCO', 'Time': Timestamp('2022-12-06 14:02:59.434656'), 'Close': 2780.4, 'Piv


-------------------------START OF 4 mins-----------------------
{'Ticker': 'ADANIPORTS', 'Time': Timestamp('2022-12-06 14:04:58.667818'), 'Close': 897.0, 'Pivot': 873.5, 'prev': 893.15}
N Prev: 893.15 max 877.25 Close 897.0

{'Ticker': 'TATAMOTORS', 'Time': Timestamp('2022-12-06 14:04:58.670782'), 'Close': 427.6, 'Pivot': 429.93333333333334, 'prev': 428.75}
N Prev: 428.75 max 434.6666666666667 Close 427.6

{'Ticker': 'AXISBANK', 'Time': Timestamp('2022-12-06 14:04:58.671657'), 'Close': 900.0, 'Pivot': 882.4499999999999, 'prev': 899.2}
N Prev: 899.2 max 891.8499999999999 Close 900.0

{'Ticker': 'BAJFINANCE', 'Time': Timestamp('2022-12-06 14:04:58.672252'), 'Close': 6687.6, 'Pivot': 6860.650000000001, 'prev': 6660.45}
N Prev: 6660.45 max 6930.5 Close 6687.6

{'Ticker': 'POWERGRID', 'Time': Timestamp('2022-12-06 14:04:58.672788'), 'Close': 221.45, 'Pivot': 223.4, 'prev': 221.9}
N Prev: 221.9 max 223.7 Close 221.45

{'Ticker': 'TATASTEEL', 'Time': Timestamp('2022-12-06 14:04:58.673303'), 

...

S Prev: 844.1 max 843.2916666666667 Close 830.8

{'Ticker': 'M&M', 'Time': Timestamp('2022-12-06 14:33:59.514628'), 'Close': 1260.9, 'Pivot': 1291.3833333333334, 'prev': 1259.8}
N Prev: 1259.8 max 1298.4916666666668 Close 1260.9

{'Ticker': 'TATACHEM', 'Time': Timestamp('2022-12-06 14:33:59.516909'), 'Close': 1044.7, 'Pivot': 1063.5, 'prev': 1056.7}
N Prev: 1056.7 max 1077.5 Close 1044.7

{'Ticker': 'ADANIENT', 'Time': Timestamp('2022-12-06 14:33:59.517880'), 'Close': 4024.5, 'Pivot': 3788.633333333333, 'prev': 3930.4}
N Prev: 3930.4 max 3853.2666666666664 Close 4024.5

{'Ticker': 'BANDHANBNK', 'Time': Timestamp('2022-12-06 14:33:59.518654'), 'Close': 244.55, 'Pivot': 229.65, 'prev': 240.95}
N Prev: 240.95 max 234.025 Close 244.55

{'Ticker': 'LTI', 'Time': Timestamp('2022-12-06 14:33:59.519239'), 'Close': 4813.05, 'Pivot': 4963.849999999999, 'prev': 4975.4}
N Prev: 4975.4 max 5026.5 Close 4813.05

{'Ticker': 'POWERGRID', 'Time': Timestamp('2022-12-06 14:33:59.519828'), 'Close': 222.15


-------------------------START OF 35 mins-----------------------
{'Ticker': 'HEROMOTOCO', 'Time': Timestamp('2022-12-06 14:35:58.472691'), 'Close': 2786.0, 'Pivot': 2762.116666666667, 'prev': 2827.65}
N Prev: 2827.65 max 2807.0083333333337 Close 2786.0

{'Ticker': 'ADANIPORTS', 'Time': Timestamp('2022-12-06 14:35:58.475509'), 'Close': 896.5, 'Pivot': 873.5, 'prev': 893.15}
N Prev: 893.15 max 877.25 Close 896.5

{'Ticker': 'BHARATFORG', 'Time': Timestamp('2022-12-06 14:35:58.476761'), 'Close': 839.65, 'Pivot': 857.4166666666666, 'prev': 847.45}
N Prev: 847.45 max 858.4833333333332 Close 839.65

{'Ticker': 'ACC', 'Time': Timestamp('2022-12-06 14:35:58.477622'), 'Close': 2622.9, 'Pivot': 2504.0, 'prev': 2621.1}
N Prev: 2621.1 max 2533.625 Close 2622.9

{'Ticker': 'IRCTC', 'Time': Timestamp('2022-12-06 14:35:58.478374'), 'Close': 716.9, 'Pivot': 740.5666666666666, 'prev': 729.1}
N Prev: 729.1 max 742.75 Close 716.9

{'Ticker': 'JUBLFOOD', 'Time': Timestamp('2022-12-06 14:35:58.479097'), '


{'Ticker': 'TECHM', 'Time': Timestamp('2022-12-06 14:36:59.248851'), 'Close': 1086.25, 'Pivot': 1060.3166666666666, 'prev': 1099.75}
N Prev: 1099.75 max 1068.6583333333333 Close 1086.25

{'Ticker': 'AXISBANK', 'Time': Timestamp('2022-12-06 14:36:59.250272'), 'Close': 899.8, 'Pivot': 882.4499999999999, 'prev': 899.2}
N Prev: 899.2 max 891.8499999999999 Close 899.8

{'Ticker': 'CONCOR', 'Time': Timestamp('2022-12-06 14:36:59.250807'), 'Close': 762.6, 'Pivot': 775.4166666666666, 'prev': 776.05}
N Prev: 776.05 max 777.15 Close 762.6

{'Ticker': 'JINDALSTEL', 'Time': Timestamp('2022-12-06 14:36:59.251232'), 'Close': 566.0, 'Pivot': 509.8666666666666, 'prev': 569.35}
N Prev: 569.35 max 522.8083333333332 Close 566.0

{'Ticker': 'KOTAKBANK', 'Time': Timestamp('2022-12-06 14:36:59.251637'), 'Close': 1931.65, 'Pivot': 1938.75, 'prev': 1938.45}
N Prev: 1938.45 max 1942.9 Close 1931.65

{'Ticker': 'BANKBARODA', 'Time': Timestamp('2022-12-06 14:36:59.252042'), 'Close': 174.1, 'Pivot': 160.85, 'pre


-------------------------START OF 41 mins-----------------------
{'Ticker': 'HAL', 'Time': Timestamp('2022-12-06 14:41:58.371092'), 'Close': 2760.75, 'Pivot': 2646.733333333333, 'prev': 2783.3}
N Prev: 2783.3 max 2700.3916666666664 Close 2760.75

{'Ticker': 'MUTHOOTFIN', 'Time': Timestamp('2022-12-06 14:41:58.373067'), 'Close': 1116.5, 'Pivot': 1086.1000000000001, 'prev': 1142.1}
N Prev: 1142.1 max 1088.775 Close 1116.5

{'Ticker': 'HDFC', 'Time': Timestamp('2022-12-06 14:41:58.374213'), 'Close': 2672.35, 'Pivot': 2624.8833333333337, 'prev': 2675.8}
N Prev: 2675.8 max 2658.716666666667 Close 2672.35

{'Ticker': 'ASIANPAINT', 'Time': Timestamp('2022-12-06 14:41:58.374777'), 'Close': 3153.4, 'Pivot': 3136.2000000000003, 'prev': 3156.9}
N Prev: 3156.9 max 3155.6750000000006 Close 3153.4

{'Ticker': 'BHARATFORG', 'Time': Timestamp('2022-12-06 14:41:58.375296'), 'Close': 839.95, 'Pivot': 857.4166666666666, 'prev': 847.45}
N Prev: 847.45 max 858.4833333333332 Close 839.95

{'Ticker': 'ACC',


-------------------------START OF 43 mins-----------------------
{'Ticker': 'AXISBANK', 'Time': Timestamp('2022-12-06 14:43:58.413387'), 'Close': 901.2, 'Pivot': 882.4499999999999, 'prev': 899.2}
N Prev: 899.2 max 891.8499999999999 Close 901.2

{'Ticker': 'CONCOR', 'Time': Timestamp('2022-12-06 14:43:58.418843'), 'Close': 763.3, 'Pivot': 775.4166666666666, 'prev': 776.05}
N Prev: 776.05 max 777.15 Close 763.3

{'Ticker': 'BANKBARODA', 'Time': Timestamp('2022-12-06 14:43:58.420601'), 'Close': 173.8, 'Pivot': 160.85, 'prev': 171.95}
N Prev: 171.95 max 163.55 Close 173.8

{'Ticker': 'SRF', 'Time': Timestamp('2022-12-06 14:43:58.421397'), 'Close': 2380.95, 'Pivot': 2414.3666666666663, 'prev': 2399.05}
N Prev: 2399.05 max 2438.7 Close 2380.95

{'Ticker': 'BHARTIARTL', 'Time': Timestamp('2022-12-06 14:43:58.422334'), 'Close': 831.95, 'Pivot': 837.8333333333334, 'prev': 844.1}
S Prev: 844.1 max 843.2916666666667 Close 831.95

{'Ticker': 'HDFCLIFE', 'Time': Timestamp('2022-12-06 14:43:59.4831